# **마을로 쇼핑몰 아이템 크롤링**
## **1 크롤링 수집함수 생성하기**
**[How to Save the Maulo Items by WGET](https://electrictoolbox.com/wget-save-different-filename/)**

**[Maulo 쇼핑목록 바로가기](http://maulo.kr/down/maulo.asp?page=1)**
```
wget http://maulo.kr/down/maulo.asp?page=1
```

In [1]:
import os
from time import sleep
from tqdm import tqdm
for page in tqdm(range(1,8)):
# for page in tqdm(range(1,2)):
    file_save = f" --output-document=./data/mauloItem{page}.html "
    terminal_text = f"http://maulo.kr/down/maulo.asp?page={page}"
#     os.system("wget " + file_save + terminal_text)
#     sleep(1)

100%|██████████| 7/7 [00:00<00:00, 50446.96it/s]


## **2 저장된 파일 내용정리**
**[Maulo Item Save by WGET](https://electrictoolbox.com/wget-save-different-filename/)**
```
wget http://maulo.kr/down/maulo.asp\?page\=1
```

In [2]:
from glob import glob
def read_tables(file_reg = './data/mauloItem?.html'):
    result = []
    files = sorted(glob(file_reg))
    for _file in files:
        with open(_file, "r") as f:
            result.append(f.readlines()[-1])
    return result

tables = read_tables()
len(tables)

7

## **3 Functional Merging**
`Image Url` 및 `내부 Tabel` 을 Text 로 저장 후 테이블 정리

In [3]:
# Table, ImageUrl 추출 및 추가
from lxml.html import fromstring, tostring
import pandas as pd

def extract_data(tables):
    result = []
    for table in tables:
        table_df = pd.read_html(table, header=0)[0]
        table_html = fromstring(table)
        # Working in the Each Table
        table_items = table_html.xpath('//table[@border="1"]/tr')[1:] # Header 제거
        result_table, result_image = [], []
        # 01 search inner table
        for item in table_items:
            item = item.xpath('.//table')
            # if item: result_table.append(tostring(item[-1], encoding="unicode"))
            if item: result_table.append(item[-1].text_content().strip())
            else:    result_table.append("")
        # 02 search Image Urls
        for number, item in enumerate(table_items):
            item = item.xpath('.//a[@target="_blank"]')
            if item:
                image_url = []
                for _ in item:
                    image_url.append("http://www.maulo.kr" + _.get('href'))
                result_image.append(image_url)
            else: result_image.append("")
        table_df.insert(9, "inTable", result_table)
        table_df.insert(11, "이미지Url", result_image)
        result.append(table_df)
    return pd.concat(result).reset_index(drop=True)

In [4]:
import re
result_df = extract_data(tables)
tokenizer = re.compile('[ ㄱ-힣0-9:.~]+')
result_df['내용'] = [ " ".join(tokenizer.findall(str(_))).strip()  
                     for _ in result_df["내용"] if _]
result_df["이미지"] = [len(_)   for _ in result_df["이미지Url"]]
result_df.to_csv('./data/maulo_items_img.csv', index=None)
result_df.head(3)

,번호,판매자,제목,상품/서비스,공급자,키워드,판매가,할인가,내용,inTable,이미지,이미지Url
0,1,(주)인스케어코어,파트서비스-욕실 클리닝,서비스,NaN,"청소 서비스, 위생 케어, 홈 클리닝, 화장실 청소, 곰팡이 제거, 인스케어",90000,90000,서비스 신청 및 고객 상담 1577 2040 평일: 09:00~ 05:3...,,2,[http://www.maulo.kr/data/2017/03/03/201703031...
1,2,(주)인스케어코어,파트서비스-주방 클리닝,서비스,NaN,"청소 서비스, 위생 케어, 홈 클리닝, 주방 청소, 기름때 제거,인스케어",90000,90000,서비스 신청 및 고객 상담 1577 2040 평일: 09:00~ 05:3...,,2,[http://www.maulo.kr/data/2017/03/03/201703031...
2,3,(주)인스케어코어,토탈서비스-360케어,서비스,NaN,"청소 서비스, 위생 케어, 홈 클리닝, 입주전 청소, 이사후 청소,인스케어",135000,135000,서비스 신청 및 고객 상담 1577 2040 평일: 09:00~ 05:3...,,2,[http://www.maulo.kr/data/2017/03/03/201703031...


## **4 Image Download**
링크에 포함된 이미지들 다운로드 실행

In [5]:
url_temp = result_df["이미지Url"]
url_temp[0][0]

'http://www.maulo.kr/data/2017/03/03/20170303131643inscare1111.jpg'

In [6]:
url_temp[0][0].replace('http://www.maulo.kr/data/', '').replace('/','_')

'2017_03_03_20170303131643inscare1111.jpg'

In [7]:
import os
from random import randint
from time import sleep
from tqdm import tqdm
file_images = []
for _imgs in tqdm(result_df["이미지Url"]):
    temp = []
    if len(_imgs) > 0:
        for _img in _imgs:        
            file_save = f" --output-document=./photo/" +\
                _img.replace('http://www.maulo.kr/data/', '').replace('/','_')
            _img = _img.replace("http://www.maulo.kr/", "http://www.nlly.net/")
            os.system("wget " + file_save + " " + _img)
            temp.append(file_save)
            sleep(randint(5,10)/10)
        file_images.append(",".join(temp))

100%|██████████| 923/923 [31:42<00:00,  2.06s/it]  


<br/>

# **Html 테이블 내용의 정리**

## **1 Html 테이블 기본 Layer 추출하기**
전체 7개의 테이블 내용 추출함수 적용하기
1. 기본 Table 에서 추가 Table, Image Link 필드 추가하기

In [ ]:
import pandas as pd
table_df = pd.read_html(tables[0], header=0)[0]
table_df.head(3)

## **2 Html 테이블 내용 추가하기**
`Image Url` 및 `내부 Tabel` 을 Text 로 저장하기

In [ ]:
from lxml.html import fromstring, tostring
table_html = fromstring(tables[0])
table_items = table_html.xpath('//table[@border="1"]/tr')[1:]
len(table_items), len(table_df)

In [ ]:
# Item 내부의 Table 내용 추출하기
count_blank, result_table = 0, []
for item in table_items:
    item = item.xpath('.//table')
    if item: result_table.append(tostring(item[-1], encoding="unicode"))
    else:    
        count_blank += 1 
        result_table.append("")
len(result_table), count_blank 

In [ ]:
result_table[39]

## **3 Html 테이블 내용 추가하기**
`Image Url` 및 `내부 Tabel` 을 Text 로 저장하기

In [ ]:
# Item 내부의 Image Url 추출하기
count_blank, result_image = 0, []
for number, item in enumerate(table_items):
    item = item.xpath('.//a[@target="_blank"]')
    if item:
        image_url = []
        for _ in item:
            image_url.append("http://www.maulo.kr" + _.get('href'))
        result_image.append(image_url)
        pass #; print(len(item))
    else:
        count_blank += 1 
        result_image.append("")

# Result Count
len(result_image), count_blank

## **4 Functional Merging**
`Image Url` 및 `내부 Tabel` 을 Text 로 저장하기

In [ ]:
from lxml.html import fromstring, tostring
import pandas as pd

def extract_data(tables):
    result = []
    for table in tables:
        table_df = pd.read_html(table, header=0)[0]
        table_html = fromstring(table)
        # Working in the Each Table
        table_items = table_html.xpath('//table[@border="1"]/tr')[1:] # Header 제거
        result_table, result_image = [], []
        # 01 search inner table
        for item in table_items:
            item = item.xpath('.//table')
            if item: result_table.append(tostring(item[-1], encoding="unicode"))
            else:    result_table.append("")
        # 02 search Image Urls
        for number, item in enumerate(table_items):
            item = item.xpath('.//a[@target="_blank"]')
            if item:
                image_url = []
                for _ in item:
                    image_url.append("http://www.maulo.kr" + _.get('href'))
                result_image.append(image_url)
            else: result_image.append("")
        table_df.insert(9, "inTable", result_table)
        table_df.insert(11, "이미지Url", result_image)
        result.append(table_df)
    return result

<br/>

# **Shopping Mall Table**

## **1 위에서 수집한 내용 본문에 덧붙이기**

In [ ]:
table_df.head(3)

In [ ]:
table_df.insert(9, "inTable", result_table)
table_df.insert(11, "이미지Url", result_image)
table_df.head(3)

In [ ]:
table_items[0].text_content().strip()

In [ ]:
table_items[1].text_content().strip()